##  Train the model with both tabular lab items and diagnosis information from discharge summaries

In [1]:
import pandas as pd

data_path = ''
# notes = pd.read_csv(note_path+'NOTEEVENTS.csv', skiprows= lambda x: x in [1289580])
lab_items = pd.read_csv(data_path+'lab_items_v01.csv')
print(lab_items.columns)

Index(['subject_id', 'hadm_id', 'admission_type', 'age', 'insurance',
       'ethnicity', 'religion', 'marital_status', 'gender', 'value',
       'valuenum', 'label', 'admittime', 'dischtime'],
      dtype='object')


In [2]:
import pandas as pd

note_path = ''
# note_path = '/media/tan/ssd-part-I/data/mimic3/1.4/not-related-to-CAD/'
# notes = pd.read_csv(note_path+'NOTEEVENTS.csv', skiprows= lambda x: x in [1289580])
notes_df = pd.read_csv(note_path+'noteevents_hf_v01.csv',low_memory=False)

In [3]:
def set_target(in_df):

    last_id = 0
    for d in in_df.index:
        if last_id == d[0]:
            # print(d)
            # (lab_items['dischtime'] - lab_items['admittime']).dt.days
            # disch_dt = np.datetime64(grouped_df.loc[(21,109451),['dischtime']].values[0])
            # admit_dt = np.datetime64(grouped_df.loc[(21,109451),['admittime']].values[0])
            dur = pd.Timedelta(in_df.loc[last_d,['dischtime']].values[0] - in_df.loc[d,['admittime']].values[0])
            if dur.days < 30:
                in_df.loc[last_d,['target']] = 1
            else:
                in_df.loc[last_d,['target']] = 0

        last_d = d
        last_id = d[0]
        
    
    return in_df

In [4]:
lab_items['dischtime'] = pd.to_datetime(lab_items['dischtime'],infer_datetime_format=True)
lab_items['admittime'] = pd.to_datetime(lab_items['admittime'],infer_datetime_format=True)

group_idx = ['subject_id', 'hadm_id', 'admission_type', 'age', 'insurance',
       'ethnicity', 'religion', 'marital_status', 'gender','label','admittime','dischtime']

grouped_df = lab_items.groupby(group_idx,as_index=False).apply(lambda x: round(x[['valuenum']].mean()))

grouped_df = grouped_df.pivot(index=['subject_id', 'hadm_id', 'admission_type', 'age', 'insurance', 'ethnicity', 'religion', 
                        'marital_status', 'gender','admittime','dischtime'],
                 columns='label',
                 values='valuenum').reset_index()

grouped_df = grouped_df.set_index(['subject_id','hadm_id'], drop = True)

grouped_df['target'] = 2

grouped_df = set_target(grouped_df.copy())

grouped_df['duration'] = (grouped_df['dischtime'] - grouped_df['admittime']).apply(lambda x: x.days)
grouped_df = grouped_df.drop(['admittime','dischtime'],axis=1)

grouped_df.fillna(0,inplace=True)

grouped_df = grouped_df.loc[~(grouped_df['target']==2)]

grouped_df

,label,admission_type,age,insurance,ethnicity,religion,marital_status,gender,"Creatinine, Serum",NTproBNP,Sodium,Urea Nitrogen,target,duration
subject_id,hadm_id,,,,,,,,,,,,,
21,109451,EMERGENCY,87.0,Medicare,WHITE,JEWISH,MARRIED,M,0.0,0.0,138.0,71.0,1,13
34,115799,EMERGENCY,300.0,Medicare,WHITE,CATHOLIC,MARRIED,M,0.0,0.0,141.0,25.0,1,1
68,108329,EMERGENCY,41.0,Medicare,BLACK/AFRICAN AMERICAN,PROTESTANT QUAKER,SINGLE,F,0.0,64499.0,134.0,40.0,0,14
105,128744,EMERGENCY,35.0,Medicare,BLACK/AFRICAN AMERICAN,PROTESTANT QUAKER,SINGLE,F,0.0,0.0,139.0,12.0,1,4
107,174162,EMERGENCY,70.0,Medicare,HISPANIC OR LATINO,CATHOLIC,SEPARATED,M,0.0,4103.0,137.0,89.0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99660,168541,EMERGENCY,75.0,Medicare,WHITE,UNOBTAINABLE,MARRIED,F,0.0,1730.0,139.0,43.0,1,4
99883,150755,EMERGENCY,73.0,Medicare,WHITE,PROTESTANT QUAKER,MARRIED,M,0.0,0.0,140.0,30.0,1,3
99897,162913,EMERGENCY,53.0,Private,BLACK/HAITIAN,7TH DAY ADVENTIST,MARRIED,M,0.0,0.0,129.0,70.0,1,1


In [5]:
notes_df = notes_df.rename(columns={'SUBJECT_ID':'pat_id','HADM_ID':'adm_id'})
notes_df = notes_df.set_index(['pat_id','adm_id'])
notes_df = notes_df.rename_axis(['subject_id','hadm_id'])
notes_df

ROW_ID   CHARTDATE  CHARTTIME  STORETIME  \
subject_id hadm_id                                             
21         109451     7237  2134-09-24        NaN        NaN   
           111970     7238  2135-02-08        NaN        NaN   
68         108329     9139  2174-01-18        NaN        NaN   
           170467     9138  2174-01-03        NaN        NaN   
105        128744     1072  2189-02-25        NaN        NaN   
...                    ...         ...        ...        ...   
99556      196292      585  2167-07-31        NaN        NaN   
99650      158023     7324  2155-01-09        NaN        NaN   
           190854     7302  2152-06-12        NaN        NaN   
99897      162913    40065  2181-08-07        NaN        NaN   
           181057    40066  2182-07-08        NaN        NaN   

                             CATEGORY DESCRIPTION  CGID  ISERROR  \
subject_id hadm_id                                                 
21         109451   Discharge summary      Report   NaN      NaN   
           111970   Discharge summary      Report   NaN      NaN   
68         108329   Discharge summary      Report   NaN      NaN   
           170467   Discharge summary      Report   NaN      NaN   
105        128744   Discharge summary      Report   NaN      NaN   
...                               ...         ...   ...      ...   
99556      196292   Discharge summary      Report   NaN      NaN   
99650      158023   Discharge summary      Report   NaN      NaN   
           190854   Discharge summary      Report   NaN      NaN   
99897      162913   Discharge summary      Report   NaN      NaN   
           181057   Discharge summary      Report   NaN      NaN   

                                                                 TEXT  Column1  
subject_id hadm_id                                                              
21         109451   Admission Date:  [**2134-9-11**]              ...      NaN  
           111970   Admission Date:  [**2135-1-30**]              ...      NaN  
68         108329   Admission Date:  [**2174-1-4**]              D...      NaN  
           170467   Admission Date:  [**2173-12-15**]             ...      NaN  
105        128744   Admission Date:  [**2189-2-21**]              ...      NaN  
...                                                               ...      ...  
99556      196292   Admission Date:  [**2167-7-30**]              ...      NaN  
99650      158023   Admission Date:  [**2155-1-2**]              D...      NaN  
           190854   Admission Date: [**2152-6-7**]        Discharg...      NaN  
99897      162913   Admission Date:  [**2181-8-6**]              D...      NaN  
           181057   Admission Date:  [**2182-7-3**]              D...      NaN  

[5385 rows x 10 columns]

In [6]:
grouped_df

,label,admission_type,age,insurance,ethnicity,religion,marital_status,gender,"Creatinine, Serum",NTproBNP,Sodium,Urea Nitrogen,target,duration
subject_id,hadm_id,,,,,,,,,,,,,
21,109451,EMERGENCY,87.0,Medicare,WHITE,JEWISH,MARRIED,M,0.0,0.0,138.0,71.0,1,13
34,115799,EMERGENCY,300.0,Medicare,WHITE,CATHOLIC,MARRIED,M,0.0,0.0,141.0,25.0,1,1
68,108329,EMERGENCY,41.0,Medicare,BLACK/AFRICAN AMERICAN,PROTESTANT QUAKER,SINGLE,F,0.0,64499.0,134.0,40.0,0,14
105,128744,EMERGENCY,35.0,Medicare,BLACK/AFRICAN AMERICAN,PROTESTANT QUAKER,SINGLE,F,0.0,0.0,139.0,12.0,1,4
107,174162,EMERGENCY,70.0,Medicare,HISPANIC OR LATINO,CATHOLIC,SEPARATED,M,0.0,4103.0,137.0,89.0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99660,168541,EMERGENCY,75.0,Medicare,WHITE,UNOBTAINABLE,MARRIED,F,0.0,1730.0,139.0,43.0,1,4
99883,150755,EMERGENCY,73.0,Medicare,WHITE,PROTESTANT QUAKER,MARRIED,M,0.0,0.0,140.0,30.0,1,3
99897,162913,EMERGENCY,53.0,Private,BLACK/HAITIAN,7TH DAY ADVENTIST,MARRIED,M,0.0,0.0,129.0,70.0,1,1


In [7]:
grouped_df = grouped_df.merge(notes_df[['TEXT']],how='left',left_on=['subject_id','hadm_id'],right_on=['subject_id','hadm_id'])
grouped_df

,,admission_type,age,insurance,ethnicity,religion,marital_status,gender,"Creatinine, Serum",NTproBNP,Sodium,Urea Nitrogen,target,duration,TEXT
subject_id,hadm_id,,,,,,,,,,,,,,
21,109451,EMERGENCY,87.0,Medicare,WHITE,JEWISH,MARRIED,M,0.0,0.0,138.0,71.0,1,13,Admission Date: [**2134-9-11**] ...
34,115799,EMERGENCY,300.0,Medicare,WHITE,CATHOLIC,MARRIED,M,0.0,0.0,141.0,25.0,1,1,NaN
68,108329,EMERGENCY,41.0,Medicare,BLACK/AFRICAN AMERICAN,PROTESTANT QUAKER,SINGLE,F,0.0,64499.0,134.0,40.0,0,14,Admission Date: [**2174-1-4**] D...
105,128744,EMERGENCY,35.0,Medicare,BLACK/AFRICAN AMERICAN,PROTESTANT QUAKER,SINGLE,F,0.0,0.0,139.0,12.0,1,4,Admission Date: [**2189-2-21**] ...
107,174162,EMERGENCY,70.0,Medicare,HISPANIC OR LATINO,CATHOLIC,SEPARATED,M,0.0,4103.0,137.0,89.0,0,3,Admission Date: [**2122-5-14**] ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99660,168541,EMERGENCY,75.0,Medicare,WHITE,UNOBTAINABLE,MARRIED,F,0.0,1730.0,139.0,43.0,1,4,NaN
99883,150755,EMERGENCY,73.0,Medicare,WHITE,PROTESTANT QUAKER,MARRIED,M,0.0,0.0,140.0,30.0,1,3,NaN
99897,162913,EMERGENCY,53.0,Private,BLACK/HAITIAN,7TH DAY ADVENTIST,MARRIED,M,0.0,0.0,129.0,70.0,1,1,Admission Date: [**2181-8-6**] D...


In [8]:
# N = 10
# dx_cols = ['dx_'+str(x) for x in range(1,N+1)]
# dx_cols


In [9]:
# N=9
# l=['abc','kel']
# l += ['nan']*(N-len(l))
# # l1 = [2,4]
# l

In [10]:
import re

grouped_df['TEXT'] = grouped_df['TEXT'].astype(str)
grouped_df['TEXT'] = grouped_df['TEXT'].str.lower()
grouped_df['TEXT'] = grouped_df['TEXT'].apply(lambda x: re.sub('[^a-z0-9-:]',' ',x)).values
grouped_df['TEXT'] = grouped_df['TEXT'].apply(lambda x: ' '.join(x.split())).values

In [11]:
sample_df = grouped_df.copy().head(5)
sample_df

,,admission_type,age,insurance,ethnicity,religion,marital_status,gender,"Creatinine, Serum",NTproBNP,Sodium,Urea Nitrogen,target,duration,TEXT
subject_id,hadm_id,,,,,,,,,,,,,,
21,109451,EMERGENCY,87.0,Medicare,WHITE,JEWISH,MARRIED,M,0.0,0.0,138.0,71.0,1,13,admission date: 2134-9-11 discharge date: 2134...
34,115799,EMERGENCY,300.0,Medicare,WHITE,CATHOLIC,MARRIED,M,0.0,0.0,141.0,25.0,1,1,nan
68,108329,EMERGENCY,41.0,Medicare,BLACK/AFRICAN AMERICAN,PROTESTANT QUAKER,SINGLE,F,0.0,64499.0,134.0,40.0,0,14,admission date: 2174-1-4 discharge date: 2174-...
105,128744,EMERGENCY,35.0,Medicare,BLACK/AFRICAN AMERICAN,PROTESTANT QUAKER,SINGLE,F,0.0,0.0,139.0,12.0,1,4,admission date: 2189-2-21 discharge date: 2189...
107,174162,EMERGENCY,70.0,Medicare,HISPANIC OR LATINO,CATHOLIC,SEPARATED,M,0.0,4103.0,137.0,89.0,0,3,admission date: 2122-5-14 discharge date: 2122...


In [12]:
grouped_df.columns

Index(['admission_type', 'age', 'insurance', 'ethnicity', 'religion',
       'marital_status', 'gender', 'Creatinine, Serum', 'NTproBNP', 'Sodium',
       'Urea Nitrogen', 'target', 'duration', 'TEXT'],
      dtype='object')

In [13]:
grouped_df.shape

(6377, 14)

In [14]:
grouped_df

,,admission_type,age,insurance,ethnicity,religion,marital_status,gender,"Creatinine, Serum",NTproBNP,Sodium,Urea Nitrogen,target,duration,TEXT
subject_id,hadm_id,,,,,,,,,,,,,,
21,109451,EMERGENCY,87.0,Medicare,WHITE,JEWISH,MARRIED,M,0.0,0.0,138.0,71.0,1,13,admission date: 2134-9-11 discharge date: 2134...
34,115799,EMERGENCY,300.0,Medicare,WHITE,CATHOLIC,MARRIED,M,0.0,0.0,141.0,25.0,1,1,nan
68,108329,EMERGENCY,41.0,Medicare,BLACK/AFRICAN AMERICAN,PROTESTANT QUAKER,SINGLE,F,0.0,64499.0,134.0,40.0,0,14,admission date: 2174-1-4 discharge date: 2174-...
105,128744,EMERGENCY,35.0,Medicare,BLACK/AFRICAN AMERICAN,PROTESTANT QUAKER,SINGLE,F,0.0,0.0,139.0,12.0,1,4,admission date: 2189-2-21 discharge date: 2189...
107,174162,EMERGENCY,70.0,Medicare,HISPANIC OR LATINO,CATHOLIC,SEPARATED,M,0.0,4103.0,137.0,89.0,0,3,admission date: 2122-5-14 discharge date: 2122...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99660,168541,EMERGENCY,75.0,Medicare,WHITE,UNOBTAINABLE,MARRIED,F,0.0,1730.0,139.0,43.0,1,4,nan
99883,150755,EMERGENCY,73.0,Medicare,WHITE,PROTESTANT QUAKER,MARRIED,M,0.0,0.0,140.0,30.0,1,3,nan
99897,162913,EMERGENCY,53.0,Private,BLACK/HAITIAN,7TH DAY ADVENTIST,MARRIED,M,0.0,0.0,129.0,70.0,1,1,admission date: 2181-8-6 discharge date: 2181-...


In [15]:
grouped_df_bkup = grouped_df.copy()

In [16]:
grouped_df = grouped_df.reset_index()

In [17]:
grouped_df

,subject_id,hadm_id,admission_type,age,insurance,ethnicity,religion,marital_status,gender,"Creatinine, Serum",NTproBNP,Sodium,Urea Nitrogen,target,duration,TEXT
0,21,109451,EMERGENCY,87.0,Medicare,WHITE,JEWISH,MARRIED,M,0.0,0.0,138.0,71.0,1,13,admission date: 2134-9-11 discharge date: 2134...
1,34,115799,EMERGENCY,300.0,Medicare,WHITE,CATHOLIC,MARRIED,M,0.0,0.0,141.0,25.0,1,1,nan
2,68,108329,EMERGENCY,41.0,Medicare,BLACK/AFRICAN AMERICAN,PROTESTANT QUAKER,SINGLE,F,0.0,64499.0,134.0,40.0,0,14,admission date: 2174-1-4 discharge date: 2174-...
3,105,128744,EMERGENCY,35.0,Medicare,BLACK/AFRICAN AMERICAN,PROTESTANT QUAKER,SINGLE,F,0.0,0.0,139.0,12.0,1,4,admission date: 2189-2-21 discharge date: 2189...
4,107,174162,EMERGENCY,70.0,Medicare,HISPANIC OR LATINO,CATHOLIC,SEPARATED,M,0.0,4103.0,137.0,89.0,0,3,admission date: 2122-5-14 discharge date: 2122...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6372,99660,168541,EMERGENCY,75.0,Medicare,WHITE,UNOBTAINABLE,MARRIED,F,0.0,1730.0,139.0,43.0,1,4,nan
6373,99883,150755,EMERGENCY,73.0,Medicare,WHITE,PROTESTANT QUAKER,MARRIED,M,0.0,0.0,140.0,30.0,1,3,nan
6374,99897,162913,EMERGENCY,53.0,Private,BLACK/HAITIAN,7TH DAY ADVENTIST,MARRIED,M,0.0,0.0,129.0,70.0,1,1,admission date: 2181-8-6 discharge date: 2181-...
6375,99982,112748,EMERGENCY,65.0,Medicare,WHITE,CATHOLIC,MARRIED,M,0.0,0.0,137.0,29.0,0,6,nan


In [18]:
dxcode_cluster_df = pd.read_csv("dxcode-cluster.csv")

In [19]:
dxcode_cluster_df.shape

(6377, 3)

In [20]:
dxcode_cluster_df.columns

Index(['Unnamed: 0', 'TEXT', 'Topic'], dtype='object')

In [21]:

#dxcode_cluster_df = dxcode_cluster_df.set_index(['subject_id','hadm_id'], drop = True)
dxcode_cluster_df

,Unnamed: 0,TEXT,Topic
0,0,"['pain', 'disease', 'lesion', 'shortness of br...",2
1,1,"['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'na...",0
2,2,"['edema', 'lactic acidosis', 'cardiomyopathy',...",4
3,3,"['fever', 'pna', 'chf', 'disease', 'febrile', ...",1
4,4,"['bleeding', 'ulcers', 'bleed', 'thrombosis', ...",1
...,...,...,...
6372,6372,"['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'na...",0
6373,6373,"['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'na...",0
6374,6374,"['hyperkalemia', 'alert', 'hypertension', 'hyp...",1
6375,6375,"['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'na...",0


In [22]:
cluster_col = dxcode_cluster_df['Topic']

In [23]:
cluster_col 

0       2
1       0
2       4
3       1
4       1
       ..
6372    0
6373    0
6374    1
6375    0
6376    0
Name: Topic, Length: 6377, dtype: int64

In [24]:
grouped_df = grouped_df.join(cluster_col)

#grouped_df_tmp.insert(1, "Topic", cluster_col)

grouped_df

,subject_id,hadm_id,admission_type,age,insurance,ethnicity,religion,marital_status,gender,"Creatinine, Serum",NTproBNP,Sodium,Urea Nitrogen,target,duration,TEXT,Topic
0,21,109451,EMERGENCY,87.0,Medicare,WHITE,JEWISH,MARRIED,M,0.0,0.0,138.0,71.0,1,13,admission date: 2134-9-11 discharge date: 2134...,2
1,34,115799,EMERGENCY,300.0,Medicare,WHITE,CATHOLIC,MARRIED,M,0.0,0.0,141.0,25.0,1,1,nan,0
2,68,108329,EMERGENCY,41.0,Medicare,BLACK/AFRICAN AMERICAN,PROTESTANT QUAKER,SINGLE,F,0.0,64499.0,134.0,40.0,0,14,admission date: 2174-1-4 discharge date: 2174-...,4
3,105,128744,EMERGENCY,35.0,Medicare,BLACK/AFRICAN AMERICAN,PROTESTANT QUAKER,SINGLE,F,0.0,0.0,139.0,12.0,1,4,admission date: 2189-2-21 discharge date: 2189...,1
4,107,174162,EMERGENCY,70.0,Medicare,HISPANIC OR LATINO,CATHOLIC,SEPARATED,M,0.0,4103.0,137.0,89.0,0,3,admission date: 2122-5-14 discharge date: 2122...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6372,99660,168541,EMERGENCY,75.0,Medicare,WHITE,UNOBTAINABLE,MARRIED,F,0.0,1730.0,139.0,43.0,1,4,nan,0
6373,99883,150755,EMERGENCY,73.0,Medicare,WHITE,PROTESTANT QUAKER,MARRIED,M,0.0,0.0,140.0,30.0,1,3,nan,0
6374,99897,162913,EMERGENCY,53.0,Private,BLACK/HAITIAN,7TH DAY ADVENTIST,MARRIED,M,0.0,0.0,129.0,70.0,1,1,admission date: 2181-8-6 discharge date: 2181-...,1
6375,99982,112748,EMERGENCY,65.0,Medicare,WHITE,CATHOLIC,MARRIED,M,0.0,0.0,137.0,29.0,0,6,nan,0


In [25]:
grouped_df = grouped_df.drop(['TEXT'],axis=1)

In [26]:
grouped_df['Topic'].value_counts()

0    3044
1    1590
2     684
3     595
4     464
Name: Topic, dtype: int64

In [27]:
grouped_df = grouped_df.drop(['subject_id', 'hadm_id'],axis=1)

### Here is the tabular data created from patient reports

In [28]:
#grouped_df.to_csv("test_text2.csv")

#grouped_df.to_csv("test_text3.csv")

In [29]:
grouped_df.shape

(6377, 14)

In [30]:
grouped_df.columns

Index(['admission_type', 'age', 'insurance', 'ethnicity', 'religion',
       'marital_status', 'gender', 'Creatinine, Serum', 'NTproBNP', 'Sodium',
       'Urea Nitrogen', 'target', 'duration', 'Topic'],
      dtype='object')

In [31]:
from sklearn.preprocessing import StandardScaler

ohe_features = ['admission_type', 'insurance', 'ethnicity', 'religion', 'marital_status', 'gender','Topic']
scaling_features = list((set(grouped_df.columns) - set(ohe_features)) - set(['target']))

scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(grouped_df[scaling_features]),columns=scaling_features,index=grouped_df.index)
df_ohe = pd.get_dummies(grouped_df[ohe_features])

tmp_df = df_scaled.merge(df_ohe,left_index=True,right_index=True)
df_final = tmp_df.merge(grouped_df['target'],left_index=True,right_index=True)

In [32]:
def predict(model, X, Y, test=True):
    import pandas as pd
    from sklearn.metrics import classification_report
    
    y_pred = pd.DataFrame(model.predict(X))
# print(ConfusionMatrixDisplay.from_predictions(Y, y_pred).confusion_matrix)
    # ConfusionMatrixDisplay.from_predictions(Y, y_pred)
    print(classification_report(Y, y_pred))
    return y_pred


def feature_importance(importance, features, top_n):
    d = {}
    for i,v in enumerate(importance):
        d[features[i]] = v
        # print('feature: %s, score: %.5f' % (features[i],v))
    d_srt = dict(sorted(d.items(), key=lambda x: x[1], reverse=True))
    
    cntr = 0
    print("Feature importance from higher to lower:")
    for k,v in d_srt.items():
        cntr += 1
        print('%s : %.2f' % (k,round(v,2)))
        if cntr > top_n:
            break

def train_and_test(X, y, epochs=100):
    from sklearn.model_selection import train_test_split
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.neural_network import MLPClassifier
    from sklearn.svm import SVC
    from sklearn.model_selection import train_test_split, GridSearchCV
    from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score, classification_report,confusion_matrix
    import time
    from xgboost import XGBClassifier
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.pipeline import Pipeline
    from sklearn.manifold import Isomap


    seed = 7
    test_size = 0.1
    x_train, x_test, y_train, y_test=train_test_split(X, y, test_size=test_size, random_state=seed)    

    models = [
            DecisionTreeClassifier(),
            LogisticRegression(),
            RandomForestClassifier(max_depth=10,min_samples_leaf=20,criterion='entropy', max_features='sqrt', n_estimators=50),
            MLPClassifier(hidden_layer_sizes=(10,30,10),activation='tanh',solver='sgd',alpha=0.0001,learning_rate='adaptive'),
            XGBClassifier(),
            # TODO: LightGBM
            # SVC(),
            # KNeighborsClassifier(n_neighbors = 15)
            ]

    for m in models:
        st=time.time()
        class_name = m.__class__.__name__
        print("#" * 50)
        print(class_name)
        print("-" * len(class_name))
        m.fit(x_train,y_train)

        print("Result on test set")
        # predict_and_report(m, x_test, y_test)
        y_pred = predict(m, x_test, y_test)

        # TODO: need to find the best model and return it back
        print("-" * 50)
        # result = report(original_data, x_test, y_test, y_pred)
        if class_name == 'LogisticRegression':
            feature_importance(m.coef_[0], x_train.columns, 50)
        elif class_name in ['DecisionTreeClassifier', 'RandomForestClassifier']:
            feature_importance(m.feature_importances_, x_train.columns, 20)    

        print("#" * 50)
        #result = report(df_h, x_test, y_test, y_pred)
        

        # if class_name == 'MLPClassifier':
        #result.to_csv(OUTPUT_PATH + class_name + '_test_set_hcode.csv' )
        #classification_report.to_csv(OUTPUT_PATH+classification_report+'report_'+m+'.csv')
        print()
        et=time.time()
        print('time_elapsed',et-st)

# TODO: torch implementation block
    # import torch
    # from common.torch_mlp import FFNModel,DNNModel

    # ffn = DNNModel(x_train.shape[1])
    # ffn.train(torch.FloatTensor(x_train.values), 
    #           torch.FloatTensor(y_train.values), 
    #           torch.FloatTensor(x_test.values), 
    #           torch.FloatTensor(y_test.values),
    #           epochs = epochs)
# torch implementation block

    return models[0] # TODO: this should be the best model

In [36]:
X = df_final.drop(['target'],axis=1)
y = pd.DataFrame(df_final['target'],columns=['target'],index=X.index)
model = train_and_test(X, y, epochs=3000)
# save_model(model,model_output_path,model_file_name)

##################################################
DecisionTreeClassifier
----------------------
Result on test set
              precision    recall  f1-score   support

           0       0.48      0.48      0.48       296
           1       0.55      0.55      0.55       342

    accuracy                           0.52       638
   macro avg       0.51      0.51      0.51       638
weighted avg       0.52      0.52      0.52       638

--------------------------------------------------
Feature importance from higher to lower:
age : 0.17
Urea Nitrogen : 0.17
duration : 0.14
Sodium : 0.13
NTproBNP : 0.05
Topic : 0.04
gender_F : 0.02
religion_CATHOLIC : 0.02
ethnicity_BLACK/AFRICAN AMERICAN : 0.02
marital_status_MARRIED : 0.02
marital_status_SINGLE : 0.02
religion_JEWISH : 0.02
religion_NOT SPECIFIED : 0.01
religion_PROTESTANT QUAKER : 0.01
ethnicity_WHITE : 0.01
marital_status_DIVORCED : 0.01
marital_status_WIDOWED : 0.01
gender_M : 0.01
admission_type_EMERGENCY : 0.01
insurance_Medic

C:\Users\sarba\anaconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sarba\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\sarba\AppData\Local\Temp\ipykernel_26248\1541421270.py:64: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel(

Result on test set
              precision    recall  f1-score   support

           0       0.54      0.23      0.33       296
           1       0.56      0.83      0.67       342

    accuracy                           0.55       638
   macro avg       0.55      0.53      0.50       638
weighted avg       0.55      0.55      0.51       638

--------------------------------------------------
Feature importance from higher to lower:
age : 0.20
Urea Nitrogen : 0.18
NTproBNP : 0.12
duration : 0.11
Sodium : 0.08
Topic : 0.06
admission_type_ELECTIVE : 0.02
admission_type_EMERGENCY : 0.02
religion_CATHOLIC : 0.02
marital_status_MARRIED : 0.02
insurance_Medicare : 0.02
insurance_Private : 0.01
gender_F : 0.01
marital_status_SINGLE : 0.01
marital_status_WIDOWED : 0.01
gender_M : 0.01
ethnicity_WHITE : 0.01
insurance_Medicaid : 0.01
religion_GREEK ORTHODOX : 0.01
religion_NOT SPECIFIED : 0.01
religion_PROTESTANT QUAKER : 0.01
##################################################

time_elapsed 0.

C:\Users\sarba\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1118: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Result on test set
              precision    recall  f1-score   support

           0       0.53      0.21      0.30       296
           1       0.55      0.84      0.66       342

    accuracy                           0.55       638
   macro avg       0.54      0.52      0.48       638
weighted avg       0.54      0.55      0.50       638

--------------------------------------------------
##################################################

time_elapsed 1.8537852764129639
##################################################
XGBClassifier
-------------
Result on test set
              precision    recall  f1-score   support

           0       0.49      0.42      0.45       296
           1       0.55      0.63      0.59       342

    accuracy                           0.53       638
   macro avg       0.52      0.52      0.52       638
weighted avg       0.53      0.53      0.52       638

--------------------------------------------------
###########################################